In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark 
findspark.init()

In [4]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder \
# .master('local[*]')\
# .appName("Iniciando com Spark")\
# .getOrCreate()

In [5]:
# spark

# Acessando o Spark UI


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
     .master('local[*]')\
    .appName("Iniciando com Spark")\
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [7]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/nrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.


In [8]:
get_ipython().system_raw('./ngrok.http 4050 &')

In [9]:
!curl -s http://localhost:4040/api/tunnels


In [10]:
spark

In [11]:
data = [('Zeca', '35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']
df = spark.createDataFrame(data, colNames)
df

DataFrame[Nome: string, Idade: string]

In [12]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
path = '/content/drive/MyDrive/curso-spark/empresas'
empresas = spark.read.csv(path, sep=';', inferSchema=True)

In [18]:
empresas.count()

4585679

In [19]:
empresas.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [20]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

In [21]:
for index, colName in enumerate(empresasColNames):
  empresas = empresas.withColumnRenamed(f"_c{index}", colName)

empresas.columns

['cnpj_basico',
 'razao_social_nome_empresarial',
 'natureza_juridica',
 'qualificacao_do_responsavel',
 'capital_social_da_empresa',
 'porte_da_empresa',
 'ente_federativo_responsavel']

In [22]:
empresas.printSchema

<bound method DataFrame.printSchema of DataFrame[cnpj_basico: int, razao_social_nome_empresarial: string, natureza_juridica: int, qualificacao_do_responsavel: int, capital_social_da_empresa: string, porte_da_empresa: int, ente_federativo_responsavel: string]>

In [23]:
path = '/content/drive/MyDrive/curso-spark/estabelecimentos'
estabelecimentos = spark.read.csv(path, sep=';', inferSchema=True)

In [24]:
path = '/content/drive/MyDrive/curso-spark/socios'
socios = spark.read.csv(path, sep=';', inferSchema=True)

In [25]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [26]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [27]:
empresas = empresas.withColumn('capital_social_da_empresa',f.regexp_replace('capital_social_da_empresa',',','.'))

In [28]:
empresas.printSchema

<bound method DataFrame.printSchema of DataFrame[cnpj_basico: int, razao_social_nome_empresarial: string, natureza_juridica: int, qualificacao_do_responsavel: int, capital_social_da_empresa: string, porte_da_empresa: int, ente_federativo_responsavel: string]>

In [29]:
empresas = empresas.withColumn('capital_social_da_empresa',empresas['capital_social_da_empresa'].cast(DoubleType()))

In [30]:
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()

,data
0,20200924
1,20201022
2,20210215


In [31]:
df.printSchema()

root
 |-- data: long (nullable = true)



In [32]:
df = df.withColumn('data', f.to_date(df.data.cast(StringType()),'yyyMMdd'))

In [33]:
df.toPandas()

,data
0,2020-09-24
1,2020-10-22
2,2021-02-15
